# Le problème : la prévision de consommation électrique

Pour garantir l'équilibre offre-demande à chaque instant et gérer l'acheminement de l'électricité, RTE construit ses propres prévisions de la consommation nationale, régionale, et locale, à différentes échéances de temps (de l'infrajournalier au pluri-annuel).

Ici on se focalise sur un problème particulier : **la prévision de la consommation électrique nationale horaire à horizon J+1**. 

## Les données : Eco2mix

La courbe de charge France est disponible sur eco2mix :
http://www.rte-france.com/fr/eco2mix/eco2mix
ou sur application mobile.

Vous pouvez naviguer sur le site pour vous familiariser avec les données sur lesquelles vous allez travailler.

## Objectif :

Au cours de cette étude de cas, nous allons aborder les différentes étapes nécessaires à la construction d'un modèle de prévision de consommation :

1) Formalisation du problème Y = f(X): que souhaite-t-on prédire (quel est mon Y) ? Avec quelles variables explicatives (quel est mon X) ?

2) Collecte des données : où se trouvent les données ? Quel est le format ? Comment les récupérer ?

3) Import des données

4) Transformation des données (feature engineering) pour entrainer et tester un premier modèle

5) Création de prévision à dire d'expert pour servir de référence

6) Découpage des données : apprentissage - test

7) Evaluer un modèle

8) Tester des algorithmes de référence : régression linéaire, forêts aléatoires

9) Itérer à partir des modèles testés pour améliorer les prévisions

Nous verrons qu'une difficulté majeure réside dans la construction des "bonnes" variables explicatives ("garbage in, garbage out").

**Le notebook est parsemé de questions (<font color='green'>en vert</font>).

## Méthodes de prévision considérées

Les modèles actuels reposent sur des méthodes de régression linéaires et non-linéaires. Nous étudierons ici les limites de la régression linéaire.

Pour améliorer les prévisions, nous aurons recours aux méthodes dites de Machine Learning. Ces méthodes ne dépendent pas d'une formalisation a priori du lien entre les variables explicatives X et la variable à expliquer Y. 
Elles sont souvent moins interprétables mais peuvent être plus efficaces en prévision. Elles peuvent nécessiter plus de temps de calcul et plus de données pour cela.

Construire un bon modèle d'apprentissage nécessite en général de la connaissance experte dans le domaine d'intérêt pour créer des modèles pertinents et efficaces. 

## To be continued : deep learning

La deuxième étude de cas permettra d'investiguer les modèles "Deep" avec réseaux de neurones, en montrant le moindre besoin en feature engineering et leur plus grande capacité à absorber l'information grâce aux représentations hiérarchiques qu'ils créent.

# Environnement de travail 

Ceci est un notebook jupyter. Il permet d'exécuter du code python, d'afficher des résultats et d'écrire du texte pour décrire l'ensemble de l'étude.

<font color='red'>
    
**NB : L'aide de python est accessible en tapant help(nom_de_la_commande)**

</font>

## Chargement des packages

In [ ]:
# Exécutez la cellule ci-dessous (par exemple avec shift-entrée)
# Si vous exécuter ce notebook depuis votre PC, il faudra peut-etre installer certaines librairies avec 
# 'pip install ma_librairie'
import os  # accès aux commandes système
import datetime  # structure de données pour gérer des objets calendaires
from datetime import timezone
import pytz
import pandas as pd  # gérer des tables de données en python
import numpy as np  # librairie d'opérations mathématiques
from math import sqrt
import zipfile  # compresser ou décompresser fichier
import requests, io #get data from url

import sklearn  # librairie de machine learning
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, iplot_mpl
import matplotlib.pyplot as plt  # tracer des visualisations
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

%autosave 0

In [ ]:
isInColab=True#False if in Jupyter

if(isInColab):#to have iplot working in colab
    import plotly.io as pio
    #pio.renderers
    pio.renderers.default = 'colab'

## Données disponibles

Choix du répertoire de travail "data_folder" dans lequel tous les fichiers csv seront entreposés. Ici le répertoire s'appelle *data*.

Ensuite on affiche les fichiers du répertoire pour vérification

In [ ]:
isDataFromGithub=False #True especially if using colab as it only download the notebook and not the entire github repository. Otherwise with Binder or in local, set it False
if isInColab:
    isDataFromGithub=True
    
if(isDataFromGithub):
    data_folder='https://raw.githubusercontent.com/rte-france/Formation_FIFA/master/data'#data depuis github
else:
    data_folder = os.path.join(os.getcwd(), "data") #data en local


In [ ]:
if(os.path.isdir(data_folder)):
    print("Mon repertoire est : {}".format(data_folder))
    print("Fichiers contenus dans ce répertoire :")
    for file in os.listdir(data_folder):
        print(" - " + file)

# Récupération des données dans Python

Dans cette partie nous allons charger les fichiers csv nécessaires pour l'analyse, puis les convertir en data-frame python : 
- y_conso_tp1.csv
- x_input_tp1.zip

NB : Les données brutes ont été pré-traitées à l'aide du notebook *preparation_donnees.ipynb* pour obtenir ces deux fichiers.

## import des données de consommation (notre Y)

In [ ]:
y_conso_csv = os.path.join(data_folder, "y_conso_tp1.csv")
y_conso = pd.read_csv(y_conso_csv)

display(y_conso.head(2)) # affichage des premières lignes
display(y_conso.tail(2)) # affichage des dernières lignes
display(y_conso.info()) # affichage du type de données et de la taille de chaque colonne

print("---")
print("Les dates sont des objets de type :")
print(type(y_conso["ds"].loc[0]))

La colonne "ds" contient la date, mais celle-ci est reconnue en tant que chaîne de caractères (https://pbpython.com/pandas_dtypes.html) et non en tant qu'objet date. 

On va la convertir en objets de type "datetime" plus appropriés pour extraire des informations comme le jour de la semaine ou l'heure.

In [ ]:
y_conso['ds'] = pd.to_datetime(y_conso['ds'], utc=True)

print(y_conso.dtypes)
display(y_conso.head(2))

print("---")
print("Les dates sont des objets de type :")
print(type(y_conso["ds"].loc[0]))

Visuellemement cela ne change rien, pour l'instant, mais plus tard ça nous facilitera la vie pour effectuer des sélections.

On peut afficher la dimension du DataFrame (toujours s'assurer que cela correspond aux valeurs attendues) : 

In [ ]:
print(y_conso.shape)

## Import des variables d'entrée du modèle prédictif 

**Attention : Les données présentes dans Xinput sont encryptées dans un fichier zip.**  
Pour les lire vous avez besoin d'un mot de passe qui ne peut vous être donné que dans le cadre d'un travail au sein de RTE.

Sinon, la lecture se déroule comme pour le fichier Yconso.csv : transformation en datetime de la colonne *ds* et vérification des dimensions.

In [ ]:
password = ""

In [ ]:
x_input_zip = os.path.join(data_folder, "x_input_tp1.zip")
if(isDataFromGithub):
    ####data sur github
    r = requests.get(x_input_zip)
    zipfile_xinput = zipfile.ZipFile(io.BytesIO(r.content))
else:
    ######data en local
    zipfile_xinput = zipfile.ZipFile(x_input_zip)

# Pour travailler avec les fichiers zip, on utilise la bibliothèque **zipfile**.
zipfile_xinput.setpassword(bytes(password, 'utf-8'))
x_input = pd.read_csv(zipfile_xinput.open('X2input.csv'), sep=",", engine='c', header=0)

x_input['ds'] = pd.to_datetime(x_input['ds'], utc=True)

Ce fichier pourrait éventuellement être enrichi à l'aide d'autres sources de données (cf. le notebook de préparation des données).

In [ ]:
print("Dimensions de X")
print(x_input.shape)

In [ ]:
print("Colonnes de X")
print(x_input.columns)

In [ ]:
print("Aperçu de X")
display(x_input.head(5))

Dans cette étude de cas, par soucis de simplicité, nous allons travailler uniquement avec des
**températures France**. Utiliser les températures des différentes stations météo serait une piste
intéressante pour améliorer le modèle.

In [ ]:
x_input = x_input[['ds', 'month', 'hour', 'posan', 'weekday', 'holiday', 'Th_real_24h_avant', 'Th_prev', 'lag1D', 'lag1W']]
display(x_input.head(5))

In [ ]:
display(x_input.info())

<font color='green'>
    
* Quelles sont les variables disponibles (dans x_input et y_conso)?
    
* Quelles sont les dimensions (nombre d’observations et de variables) de x_input et y_conso après lecture des fichiers csv? Est-ce cohérent?

* Les données présentes dans x_input vous semblent-elles pertinentes pour prédire la consommation nationale présente dans y_conso ?

* Que pensez-vous de cette notion de "Température France"?
</font>

# Visualisation des données 

La DataScience et le Machine Learning supposent de bien appréhender les données sur lesquelles nos modèles vont être entrainés. Pour cela, il est utile de faire des statistiques descriptives et des visualisations de nos différentes variables.

Traitant d'un problème de prévision, on visualisera en particulier des séries temporelles.

--> cf formation DATASX

Réaliser quelques statistiques descriptives en préambule peut mettre en évidence des variables explicatives dans le cas d'étude d'un problème jamais traité.

# Outils de construction de modèle
<img src="pictures/etabli.jpg"  width=500 >

## Construction des jeux d'entrainement et de test

Pour éviter de construire un modèle qui apprend "par coeur" sur ses données, et qui disposerait alors d'une capacité de généralisation faible, il est d'usage courant de disposer de plusieurs jeux de données (de caractéristiques similaires). Le minimum est de construire 

* un jeu d'entraînement, sur lequel on cale les paramètres du modèle, 

* un jeu de test, jamais vu durant l'entraînement, sur lequel on va évaluer le modèle. 

Rapidement dit : un bon modèle est un modèle dont la capacité prédictive ne se dégrade pas trop sur le jeu test.

Pour cela, on crée la fonction *prepareDataSetEntrainementTest* qui va permettre de couper Y et Xinput en deux parties.

In [ ]:
def prepareDataSetEntrainementTest(x, y, date_debut, date_rupture, nb_jours_lag_regresseur=0):
    
    dateStart = x.iloc[0]['ds']
    
    # si un a un regresseur avec du lag, il faut prendre en compte ce lag 
    # et commencer l'entrainement a la date de debut des donnees+ce lag
    DateStartWithLag = dateStart + pd.Timedelta(str(nb_jours_lag_regresseur) + ' days')
    
    x_test = x[(x.ds >= date_rupture)]
    x_train = x[(x.ds < date_rupture) & (x.ds > DateStartWithLag) & (x.ds > date_debut)]
    y_train = y[(y.ds < date_rupture) & (y.ds > DateStartWithLag) & (y.ds > date_debut)]
    y_test = y[(x.ds >= date_rupture)]
    
    return x_train, x_test, y_train, y_test

## Fonctions utilitaires

Créons la fonction modelError qui va calculer pour un échantillon (y, y_hat) différents scores :
- erreur relative moyenne (MAPE en %)
- erreur relative max (en %)
- RMSE (en MW)

Cette fonction est ensuite utilisée par les fonctions *evaluation* et *evaluation_par* qui nous permettront d'évaluer nos modèles.

<img src="pictures/evaluation.jpg"  width=250 >

In [ ]:
def modelError(y_true, y_hat):

    Y = y_true.reset_index(drop=True).copy()
    
    relative_errors = np.abs((Y['y'] - y_hat) / Y['y']) 
    mean_error = np.mean(relative_errors)
    max_error = np.max(relative_errors)
    rmse = np.sqrt(mean_squared_error(Y['y'], y_hat))
   
    return relative_errors, mean_error, max_error, rmse

In [ ]:
def evaluation(y_train, y_test, y_train_hat, y_test_hat):
    # Ytrain et Ytest ont deux colonnes : ds et y
    # YtrainHat et YTestHat sont des vecteurs
    
    ErreursTest, ErreurMoyenneTest, ErreurMaxTest, RMSETest = modelError(y_test, y_test_hat)
    print("l'erreur relative moyenne de test est de :" + str(round(ErreurMoyenneTest * 100, 1)) + "%")
    print("l'erreur relative max de test est de :" + str(round(ErreurMaxTest * 100, 1)) + "%")
    print('le rmse de test est de :' + str(round(RMSETest, 0)))
    print()
    
    ErreursTest, ErreurMoyenneTest, ErreurMaxTest, RMSETest = modelError(y_train, y_train_hat)
    print("l'erreur relative moyenne de train est de :" + str(round(ErreurMoyenneTest * 100, 1)) + "%")
    print("l'erreur relative max de train est de :" + str(round(ErreurMaxTest * 100, 1)) + "%")
    print('le rmse de test est de :' + str(round(RMSETest, 0))) 

In [ ]:
def evaluation_par(X, Y, Yhat, avec_JF=True):
    Ytmp = Y.copy()
    Ytmp['weekday'] = Ytmp.ds.dt.weekday
    Ytmp['hour'] = Ytmp.ds.dt.hour
    if(avec_JF):
        Ytmp['jours_feries'] = X['jours_feries'].values
    Ytmp['APE'] = np.abs(Ytmp['y'] - Yhat) / Ytmp['y']
    dataWD = Ytmp[['weekday', 'APE']]
    groupedWD = dataWD.groupby(['weekday'], as_index=True)
    statsWD = groupedWD.aggregate([np.mean])
    dataHour = Ytmp[['hour', 'APE']]
    groupedHour = dataHour.groupby(['hour'], as_index=True)
    statsHour = groupedHour.aggregate([np.mean])
    
    if(avec_JF):
        dataJF = Ytmp[['jours_feries', 'APE']].copy()
        groupedJF = dataJF.groupby(['jours_feries'], as_index=True)
        statsJF = groupedJF.aggregate([np.mean])
    else:
        statsJF = None
    
    return statsWD, statsHour, statsJF

# Préparation des variables explicatives : préparation de Xinput

L'objectif de cette partie est d'enrichir Xinput à partir des données initiales. Il s'agit notamment d'exploiter les différentes informations calendaires disponibles.

On encode les données calendaires en **one-hot encoding** pour le modèle. Autrement dit, on construit pour chaque modalité, une variable binaire associée.
Cet encodage est nécessaire pour que le modèle mathématique puisse appréhender la notion de date.

In [ ]:
encoded_weekday = pd.get_dummies(x_input['weekday'], prefix="weekday")
encoded_month = pd.get_dummies(x_input['month'], prefix="month")
encoded_hour = pd.get_dummies(x_input['hour'], prefix="hour")

In [ ]:
encoded_weekday.head(3)

In [ ]:
encoded_month.head(3)

In [ ]:
encoded_hour.head(3)

In [ ]:
x_input = pd.concat([x_input, encoded_month, encoded_weekday, encoded_hour], axis=1)
x_input = x_input.drop(columns=['month', 'weekday', 'hour'])

In [ ]:
print(x_input.shape)
print(x_input.columns)

On crée une variable binaire associée à chaque jour férié.

In [ ]:
#Récupération des jours fériés dans Xinput
encoded_holidays = pd.get_dummies(x_input[['holiday']], prefix="JF")
encoded_holidays['jours_feries'] = encoded_holidays.sum(axis=1)
x_input = pd.concat([x_input, encoded_holidays], axis=1)
x_input = x_input.drop(['holiday'], axis=1)

On ajoute des températures seuillées, à 15°C pour l'effet chauffage, et à 18°C pour l'effet climatisation.

In [ ]:
threshold_temperature_heat = 15
threshold_temperature_cool = 18

x_input['temp_prev_with_threshold_heat'] = np.maximum(0, threshold_temperature_heat - x_input['Th_prev'].values)
x_input['temp_prev_with_threshold_cool'] = np.maximum(0, x_input['Th_prev'].values - threshold_temperature_cool)

In [ ]:
# Affichage de toutes les variables de base
list(x_input)  # list plutôt que print pour avoir la liste complète

Enfin, nous construisons les listes pour appeler plus rapidement les colonnes d'un même type.

In [ ]:
cols_to_keep_weather = [s for s in x_input.columns if 'Th_prev' in s]
cols_to_keep_month = [v for v in x_input.columns if 'month' in v]
cols_to_keep_weekday = [v for v in x_input.columns if 'weekday' in v]
cols_to_keep_hour = [v for v in x_input.columns if 'hour' in v]
cols_to_keep_holidays = [v for v in x_input.columns if 'JF_' in v]

In [ ]:
# on souhaite un jeu de test qui commence à partir du 1er mai 2017
date_debut = pytz.utc.localize( datetime.datetime(year=2014, month=1, day=15))  # pour éviter les NaN dans le jeu de données
date_rupture = pytz.utc.localize(datetime.datetime(year=2017, month=12, day=1))  # début du challenge prevision de conso
nb_jours_lag_regresseur = 0

In [ ]:
y_conso.tail()

In [ ]:
x_train, x_test, y_train, y_test = prepareDataSetEntrainementTest(
    x_input, 
    y_conso, 
    date_debut, 
    date_rupture, 
    nb_jours_lag_regresseur
)

In [ ]:
print('La taille de l échantillon XinputTrain est : ' + str(x_train.shape))
print('La taille de l échantillon XinputTest est : ' + str(x_test.shape))
print('La taille de l échantillon YconsoTrain est : ' + str(y_train.shape))
print('La taille de l échantillon YconsoTest est : ' + str(y_test.shape))
print("La proportion de data d'entrainement est de : " + str(round(y_train.shape[0] / (y_train.shape[0] + y_test.shape[0]), 2)) + "%")

# <b style="color:red;"> Fin du TP ici. </b>

# Partie optionnelle : 

# Construction d'un modèle prédictif naïf

<img src="pictures/hommeNaif.png" width=500 >

## Une idée de modèle naïf : on plaque bêtement la valeur de consommation nationale de la veille
Ce modèle n'a même pas besoin de regarder le jeu d'entrainement !

In [ ]:
pred_train_naif_1 = x_train["lag1D"]
pred_test_naif_1 = x_test["lag1D"]

evaluation(y_train, y_test,  pred_train_naif_1.values, pred_test_naif_1.values)

Bon c'est pas fou...

# Modèle Type Système Expert (boîte blanche)

Chez RTE, on considère qu'une baisse moyenne de 1°C conduit à une augmentation de 2400MW de la consommation nationale pour des températures inférieures à 15°C. On propose donc comme consommation prévue la consommation de la veille, corrigée par 2400 fois l'écart à la température de la veille, si l'on n'excède pas les 15°C.


<img src="pictures/ExpertJamy.jpg" width=500 >

## Definition des régles expertes

In [ ]:
# Petit rappel
print(threshold_temperature_heat)

In [ ]:
# C'est ici que l'on fait intervenir notre expertise !
delta_MW_par_degre = 2400

In [ ]:
# prévision train
temp_prev_with_threshold = np.minimum([threshold_temperature_heat], x_train['Th_prev'].values)
temp_actual_with_threshold = np.minimum([threshold_temperature_heat], x_train['Th_real_24h_avant'].values)

delta_temp = temp_prev_with_threshold - temp_actual_with_threshold
delta_MW_because_temp = delta_temp * delta_MW_par_degre

pred_train_naif_2 = x_train["lag1D"] - delta_MW_because_temp

# prévision test
temp_prev_with_threshold = np.minimum([threshold_temperature_heat], x_test['Th_prev'].values)
temp_actual_with_threshold = np.minimum([threshold_temperature_heat], x_test['Th_real_24h_avant'].values)

delta_temp = temp_prev_with_threshold - temp_actual_with_threshold
delta_MW_because_temp = delta_temp * delta_MW_par_degre
pred_test_naif_2 = x_test["lag1D"] - delta_MW_because_temp

# scores
evaluation(y_train, y_test,  pred_train_naif_2.values, pred_test_naif_2.values)

Bon... Bien essayé avec ces modèles bricolés, mais maintenant on va être plus sérieux !

# Apprentissage Automatique


## Régression linéaire simple (boîte grise)

Le modèle naïf avec expertise métier a été inspiré de la forme de la courbe d'évolution de la consommation en fonction de la température en France. 

In [ ]:
plt.scatter(x_input['Th_prev'], y_conso['y'], alpha=0.2)
plt.show()

La consommation pourrait être modélisée par une fonction linéaire par morceaux de la température, avec une pente plus importante pour les températures froides que pour les températures élevées. Au lieu de fixer les gradients à 2400MW/°C et 0, ceux-ci pourraient être calibrés à partir des données.


### Entrainer un modèle
Notre modèle a des paramètres qu'il va falloir maintenant apprendre au vu de notre jeu d'entrainement. Il faut donc caler notre modèle sur ce jeu d'entrainement.

In [ ]:
cols_LR = ["Th_prev", 'temp_prev_with_threshold_heat', 'temp_prev_with_threshold_cool']
# cols_LR = ["Th_prev", 'temp_prev_with_threshold_heat', 'temp_prev_with_threshold_cool'] + cols_to_keep_month + cols_to_keep_hour
# cols_LR = ["Th_prev", 'temp_prev_with_threshold_heat', 'temp_prev_with_threshold_cool'] + cols_to_keep_month + cols_to_keep_hour + cols_to_keep_weekday
print(cols_LR)

linear_regr = linear_model.LinearRegression()
linear_regr.fit(x_train[cols_LR], y_train[['y']])

### Interpréter le modèle 

In [ ]:
coefs_jolis = pd.DataFrame(
    np.concatenate((np.array([cols_LR]).T, linear_regr.coef_.T), axis=1),
    columns = ['variable', 'coefficient']
)
coefs_jolis.sort_values(by="coefficient", ascending=False)

display(coefs_jolis)

<font color='green'>

* Commentez les coefficients de régression obtenus. 
* Comparez notamment les gradients obtenus avec le modèle naïf.

</font>

### Faire des prédictions
Une fois qu'un modèle de prévision est entrainé, il ne s'avère utile que s'il est performant sur de nouvelles situations. Faisons une prévision sur notre jeu de test. Traçons les courbes obtenues et calculons les scores.

In [ ]:
forecast_train = np.concatenate(linear_regr.predict(x_train[cols_LR]))
forecast_test = np.concatenate(linear_regr.predict(x_test[cols_LR]))

In [ ]:
plt.scatter(forecast_test, y_test[['y']])
plt.plot([y_test[['y']].min(), y_test[['y']].max()], [y_test[['y']].min(), y_test[['y']].max()], 'r', lw=2)
plt.show()

plt.plot(y_test['ds'], y_test['y'], 'b', y_test['ds'], forecast_test, 'r')
plt.show()

plt.scatter(x_test['Th_prev'], y_test['y'], alpha=0.2)
plt.scatter(x_test['Th_prev'], forecast_test, alpha=0.2)
plt.show()

In [ ]:
evaluation(y_train, y_test, forecast_train,  forecast_test)
eval_weekday, eval_hour, eval_JF = evaluation_par(x_test, y_test, forecast_test, avec_JF=True)

display(round(eval_weekday * 100, 1))
display(round(eval_hour * 100, 1))
display(round(eval_JF * 100, 1))

#### Comment se distribue l'erreur ?

In [ ]:
erreur_relative_test, erreur_moyenne_test, erreur_max_test, rmse = modelError(y_test, forecast_test)

In [ ]:
num_bins = 100
plt.hist(erreur_relative_test, num_bins)
plt.show()

#### A quel moment se trompe-t-on le plus ?

In [ ]:
iplot([{"x": y_test['ds'], "y": erreur_relative_test}])

Regardons les erreurs les plus flagrantes :

In [ ]:
q = 0.9975
threshold = np.quantile(erreur_relative_test.values, q)

print("Quantile des erreurs à {}% : {}".format(q * 100,  round(threshold, 5)))

In [ ]:
error_order = np.argsort(erreur_relative_test.values)[::-1]
mask = (erreur_relative_test.iloc[error_order] >= threshold)
erreurs_df = pd.DataFrame(
    np.concatenate((y_test[['ds','y']],
                    y_test.ds.dt.weekday.values.reshape(-1,1),
                    y_test.ds.dt.hour.values.reshape(-1,1),
                    100*erreur_relative_test.values.reshape(-1,1),
                    np.array([forecast_test]).T), axis=1),
    columns=["date","y", "weekday", "hour","erreur_relative (%)","prev"]
)[["date","weekday", "hour","y","prev","erreur_relative (%)"]]

display(erreurs_df.iloc[error_order][mask])

In [ ]:
y_test_copy = y_test[['ds','y']].copy()
y_test_copy["erreur_relative_%"] = erreur_relative_test.values * 100
y_test_copy['weekday'] = y_test_copy.ds.dt.weekday
y_test_copy['hour'] = y_test_copy.ds.dt.hour
y_test_copy['is_bank_holiday'] = x_test["jours_feries"]

groupe_wd = y_test_copy.groupby(['weekday'], as_index=True)
stats_wd = groupe_wd["y", "erreur_relative_%"].aggregate(["mean"])

groupe_hour = y_test_copy.groupby(['hour'], as_index=True)
stats_hour = groupe_hour["y", "erreur_relative_%"].aggregate(["mean"])

groupe_bh = y_test_copy.groupby(['is_bank_holiday'], as_index=True)
stats_bh = groupe_bh["y", "erreur_relative_%"].aggregate(["mean"])

display(stats_wd)
display(stats_hour)
display(stats_bh)

Au vu des résultats précédents :
<font color= 'green'>

- que pensez-vous du modèle?
- comment se distribue l'erreur?
- quand se trompe-t-on le plus?
- quelles variables explicatives ajouter?

Vous pouvez aussi réessayer avec un cols_LR qui contient les mois et les heures.

</font>

## Modèles avancés (boîtes noires)

### Modèle RandomForest

<img src="pictures/randomForestExplain.png" width=500>

#### Choix des données d'entrée

In [ ]:
cols_RF = ['lag1D','lag1W']\
+ cols_to_keep_weather\
+ cols_to_keep_month\
+ cols_to_keep_weekday\
+ cols_to_keep_hour\
+ cols_to_keep_holidays

list(cols_RF)

In [ ]:
display(x_train[cols_RF].head(20))

#### Entrainement du modèle

In [ ]:
# La cellule peut prendre un peu de temps à exécuter
rf_regr = RandomForestRegressor(
    n_estimators=30, 
    max_depth=20,
    n_jobs=3,
)

rf_regr.fit(x_train[cols_RF], y_train['y'])

<font color='green'>

* Grâce à l'aide de la fonction, expliquer les paramètres de cette méthode

</font>

#### Prediction

In [ ]:
forecast_train = rf_regr.predict(x_train[cols_RF])
forecast_test = rf_regr.predict(x_test[cols_RF])

#### Evaluation

In [ ]:
evaluation(y_train, y_test, forecast_train, forecast_test)

# on visualise nos previsions par rapport a la realité
plt.plot(y_test['ds'], y_test['y'], 'b', y_test['ds'], forecast_test, 'r')
plt.show()

In [ ]:
eval_weakday, eval_hour, eval_JF = evaluation_par(x_test, y_test, forecast_test)

display(round(eval_weakday * 100, 1))
display(round(eval_hour * 100, 1))
display(round(eval_JF * 100, 1))

In [ ]:
erreur_relative_test, erreur_moyenne_test, erreur_max_test, rmse = modelError(y_test, forecast_test)

In [ ]:
num_bins = 100
plt.hist(erreur_relative_test, num_bins)
plt.show()

#### A quel moment se trompe-t-on le plus ?

In [ ]:
iplot([{"x": y_test['ds'], "y": erreur_relative_test}])

In [ ]:
error_order = np.argsort(erreur_relative_test.values)[::-1]

erreurs_df = pd.DataFrame(
    np.concatenate((y_test[['ds','y']],
                    y_test.ds.dt.weekday.values.reshape(-1,1),
                    y_test.ds.dt.hour.values.reshape(-1,1),
                    100*erreur_relative_test.values.reshape(-1,1),
                    np.array([forecast_test]).T), axis=1),
    columns=["date","y", "weekday", "hour","erreur_relative (%)","prev"]
)[["date","weekday", "hour","y","prev","erreur_relative (%)"]]

display(erreurs_df.iloc[error_order].head(20))

# Bonus: à vous de jouer

Bravo ! Vous avez déjà créé un premier modèle performant pour faire des prévisions sur une fenêtre glissante à horizon 24h !

Maintenant à vous de mettre votre expertise pour améliorer les performances de vos modèles. Vous pouvez continuer à explorer le problème selon plusieurs axes:
- choix du modèle (xgboost ?), tuning
- apprendre votre modèle sur une période différente
- créer de nouvelles variables explicatives ? Quid de la météo et de la température? Des jours fériés ? Du feature engineering plus complexe...
- détecter des outliers dans les données

Si vous voulez en apprendre plus => direction la formation DATAS2 !

Mettez-vous en 3 groupes, explorez pendant 30 minutes, et restituez.